## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-05-10-54-40 +0000,nyt,10 People Convicted of Cyberbullying Brigitte ...,https://www.nytimes.com/2026/01/05/world/europ...
1,2026-01-05-10-48-00 +0000,wapo,"Trump revives an old vision of American power,...",https://www.washingtonpost.com
2,2026-01-05-10-48-00 +0000,wapo,Live updates: Maduro and wife to be arraigned ...,https://www.washingtonpost.com/world/2026/01/0...
3,2026-01-05-10-47-00 +0000,wsj,Oil Prices Fall as U.S. Capture of Venezuela’s...,https://www.wsj.com/business/energy-oil/asian-...
4,2026-01-05-10-46-36 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2026/01/05/world/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2344/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
24,venezuela,62
7,trump,41
16,maduro,39
157,new,24
23,capture,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
152,2026-01-04-20-33-28 +0000,nypost,Trump threatens Venezuela’s new leader Delcy R...,https://nypost.com/2026/01/04/us-news/trump-th...,192
193,2026-01-04-15-22-48 +0000,nyt,How Trump Fixed On a Maduro Loyalist as Venezu...,https://www.nytimes.com/2026/01/04/world/ameri...,179
62,2026-01-05-04-57-44 +0000,nypost,"Sara Foster praises Trump’s capture of Maduro,...",https://nypost.com/2026/01/04/us-news/sara-fos...,163
30,2026-01-05-09-20-28 +0000,wapo,Trump says U.S. is ‘in charge’ in Venezuela af...,https://www.washingtonpost.com/world/2026/01/0...,159
20,2026-01-05-10-00-26 +0000,wapo,"What to know about Maduro, his capture and Tru...",https://www.washingtonpost.com/world/2026/01/0...,157


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
152,192,2026-01-04-20-33-28 +0000,nypost,Trump threatens Venezuela’s new leader Delcy R...,https://nypost.com/2026/01/04/us-news/trump-th...
198,48,2026-01-04-14-45-20 +0000,nypost,Marco Rubio says Venezuela’s VP Delcy Rodrígue...,https://nypost.com/2026/01/04/us-news/marco-ru...
161,46,2026-01-04-19-53-22 +0000,nypost,Owner of Swiss nightclub where deadly fire bro...,https://nypost.com/2026/01/04/world-news/owner...
92,37,2026-01-05-01-24-33 +0000,nypost,"6-year-old girl, 90-year-old passenger killed ...",https://nypost.com/2026/01/04/us-news/6-year-o...
3,36,2026-01-05-10-47-00 +0000,wsj,Oil Prices Fall as U.S. Capture of Venezuela’s...,https://www.wsj.com/business/energy-oil/asian-...
94,31,2026-01-05-01-05-44 +0000,nypost,Far-left extremist attack on Berlin power grid...,https://nypost.com/2026/01/04/world-news/far-l...
120,28,2026-01-04-23-19-44 +0000,nypost,Freed Israeli hostage details fear of being ma...,https://nypost.com/2026/01/04/world-news/freed...
106,22,2026-01-05-00-14-31 +0000,nypost,LI commuters hit from all sides as MTA jacks u...,https://nypost.com/2026/01/04/us-news/li-commu...
67,21,2026-01-05-04-29-58 +0000,nypost,Trump renews calls to takeover Greenland — as ...,https://nypost.com/2026/01/04/world-news/whos-...
179,21,2026-01-04-18-09-40 +0000,nypost,Satellite images reveal damage from US strikes...,https://nypost.com/2026/01/04/world-news/satel...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
